# About this Kernel
This is my first attempt to publish a kernel in Kaggle. I will try H2O AutoML  on this IEEE Fraud transaction dataset. Primary goal for me is to work on a dataset that has combination of features (numbers, categorical, datetime) and the features are not explanatory, so i could experiment with pre-processing & Feature engineering concepts


## Credits
1. Thanks to kernels submitted by tunguz. I have used some of the concepts in this Kernel
https://www.kaggle.com/tunguz/ieee-with-h2o-automl


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import gc
import itertools

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='12G')

3.26.0.2
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_222"; OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1~deb9u1-b10); OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp67tep73u
  JVM stdout: /tmp/tmp67tep73u/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp67tep73u/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_unknownUser_ra237c
H2O cluster total nodes:,1
H2O cluster free memory:,10.67 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
train_transaction=h2o.import_file("../input/train_transaction.csv")
train_identity=h2o.import_file("../input/train_identity.csv")
test_transaction=h2o.import_file("../input/test_transaction.csv")
test_identity=h2o.import_file("../input/test_identity.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train= train_transaction.merge(train_identity, all_x=True)
test= test_transaction.merge(test_identity, all_x=True)

In [5]:
train.shape, train_transaction.shape,train_identity.shape, test.shape, test_transaction.shape, test_identity.shape

((590540, 434),
 (590540, 394),
 (144233, 41),
 (506691, 433),
 (506691, 393),
 (141907, 41))

# Let's treat the TransactionDT feature, per the dataset description given
References: https://www.kaggle.com/wajihullahbaig/up-sampling-on-every-hour

In [6]:
#Creates an hour of the day feature, encoded as 0-23. 
train.hours=((train['TransactionDT']/(3600)).floor())%24
test.hours=((test['TransactionDT']/(3600)).floor())%24

# Drop TransactionDT

In [7]:
X_train = train.drop(['TransactionDT'], axis=1)
X_test = test.drop(['TransactionDT'], axis=1)
del train_transaction, train_identity, test_transaction, test_identity
del train, test

# Find & tag categorical variables as H2O factor

In [8]:
for f in X_train.columns:
    if X_train[f].columns_by_type(coltype='categorical'): 
        X_train[f]=X_train[f].asfactor()

In [9]:
for f in X_test.columns:
    if X_test[f].columns_by_type(coltype='categorical'): 
        X_test[f]=X_test[f].asfactor()

In [10]:
X_train.pop('TransactionID')
X_test.pop('TransactionID')

TransactionID
3.66355e+06
3.66355e+06
3.66355e+06
3.66355e+06
3.66355e+06
3.66355e+06
3.66356e+06
3.66356e+06
3.66356e+06
3.66356e+06


In [11]:
x=X_train.columns[1:]
y='isFraud'
X_train[y]=X_train[y].asfactor()


# Lets try H2O autoML

In [12]:
%%time
aml = H2OAutoML(max_models=20, seed=18, max_runtime_secs=18000, stopping_rounds=5,stopping_metric="aucpr",stopping_tolerance=1e-2)
aml.train(x=x, y=y, training_frame=X_train)

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Field = stopping_metric cannot be set to value = aucpr
  Request: POST /99/AutoMLBuilder
    json: {'input_spec': {'response_column': 'isFraud', 'training_frame': 'py_933_sid_be32', 'ignored_columns': []}, 'build_control': {'stopping_criteria': {'max_runtime_secs': 18000, 'max_models': 20, 'stopping_metric': 'aucpr', 'stopping_tolerance': 0.01, 'stopping_rounds': 5, 'seed': 18}, 'nfolds': 5, 'keep_cross_validation_predictions': False, 'keep_cross_validation_models': False, 'keep_cross_validation_fold_assignment': False, 'project_name': 'automl_py_933_sid_be32'}, 'build_models': {'exclude_algos': None}}


In [13]:
%%time
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

Parse progress: |█████████████████████████████████████████████████████████| 100%
CPU times: user 44 ms, sys: 8 ms, total: 52 ms
Wall time: 69.8 ms
This H2OFrame is empty.


In [14]:
%%time
aml.leader

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 18.4 µs


In [15]:
%%time
y_preds = aml.predict(X_test)
y_preds['p1'].as_data_frame().values.flatten().shape

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [16]:
%%time
import pandas as pd
sample_submission = pd.read_csv('../input/sample_submission.csv')

CPU times: user 100 ms, sys: 92 ms, total: 192 ms
Wall time: 193 ms


In [17]:
%%time
sample_submission['isFraud'] = y_preds['p1'].as_data_frame().values
sample_submission.to_csv('H2O_AutoML_submission_1.csv', index=False)

NameError: name 'y_preds' is not defined